In [1]:
pip install msrest

     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 41 kB 575 kB/s 


In [2]:
pip install requests

In [3]:
pip install Pillow

In [4]:
pip install azure-cognitiveservices-vision-computervision

In [2]:
import os
import io
from  msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes,VisualFeatureTypes
import requests
from PIL import Image,ImageDraw,ImageFont
import json 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import re
import sys
import time

In [3]:
list1=[]
Data=[]

In [4]:

#Code for making ORB ed image
def orbmaker(reference,actualimage):
    im1=cv2.imread(reference,cv2.IMREAD_COLOR)
    im1=cv2.cvtColor(im1,cv2.COLOR_BGR2RGB)
    im2=cv2.imread(actualimage,cv2.IMREAD_COLOR)
    im2=cv2.cvtColor(im2,cv2.COLOR_BGR2RGB)
    im1_gray=cv2.cvtColor(im1,cv2.COLOR_BGR2GRAY)
    im2_gray=cv2.cvtColor(im2,cv2.COLOR_BGR2GRAY)
    maxfeatures=500
    orb=cv2.ORB_create(maxfeatures)
    keypoints1,descriptors1=orb.detectAndCompute(im1_gray,None)
    keypoints2,descriptors2=orb.detectAndCompute(im2_gray,None)
    im1_display=cv2.drawKeypoints(im1,keypoints1,outImage=np.array([]),color=(255,0,0),flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT)
    im2_display=cv2.drawKeypoints(im2,keypoints2,outImage=np.array([]),color=(255,0,0),flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT)
    matcher=cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches=matcher.match(descriptors1,descriptors2,None)
    im_matches=cv2.drawMatches(im1,keypoints1,im2,keypoints2,matches,None)
    points1=np.zeros((len(matches),2),dtype=np.float32)
    points2=np.zeros((len(matches),2),dtype=np.float32)
    for i,match in enumerate(matches):
        points1[i,:]=keypoints1[match.queryIdx].pt
        points2[i,:]=keypoints2[match.trainIdx].pt
        h,mask=cv2.findHomography(points2,points1,cv2.RANSAC)
    height,width,channels=im1.shape
    im2_reg=cv2.warpPerspective(im2,h,(width,height))
    return im2_reg

In [5]:
#Mirror image resolver

def read_this(image_file, gray_scale=False):
    image_src = cv2.imread(image_file)
    if gray_scale:
        image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    else:
        image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    return image_rgb

def mirror_this(image_file, gray_scale=False, with_plot=False):
    image_rgb = read_this(image_file=image_file, gray_scale=gray_scale)
    image_mirror = np.fliplr(image_rgb)
    if with_plot:
        fig = plt.figure(figsize=(10, 20))
        ax1 = fig.add_subplot(2, 2, 1)
        ax1.axis("off")
        ax1.title.set_text('Original')
        ax2 = fig.add_subplot(2, 2, 2)
        ax2.axis("off")
        ax2.title.set_text("Mirrored")
        if not gray_scale:
            ax1.imshow(image_rgb)
            ax2.imshow(image_mirror)
        else:
            ax1.imshow(image_rgb, cmap='gray')
            ax2.imshow(image_mirror, cmap='gray')
        return True
    return image_mirror

In [6]:
#Facedetector Code
def facedetector(image_file):
    Image=cv2.imread(image_file)
    croppedimage = Image[250:600,600:900]
    gray = cv2.cvtColor(croppedimage, cv2.COLOR_BGR2GRAY)

    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(30, 30)
    )

    return len(faces)

In [7]:
#Function to convert image into text
def ocrmaker(image_file):

    # api  credential 
    credential=json.load(open('credential.json'))
    API_KEY=credential['subscription_key']
    ENDPOINT=credential['endpoint']
    cv_client=ComputerVisionClient(ENDPOINT,CognitiveServicesCredentials(API_KEY))

    #res=cv_client.read(url='https://1.bp.blogspot.com/-smHHgvypUDw/XxbkVjGqn3I/AAAAAAABwAk/MhFzTZIvwrAQkVdTT59sLVb-0A3PIEgfQCLcBGAsYHQ/s1600/Convert%2BHandwritten%2BNotes%2BInto%2BGoogle%2BDocs.png',language="en",raw=True)
    res=cv_client.read_in_stream(open(image_file,'rb'),raw=True,language='en', model_version="2022-01-30-preview")
    operation_loc=res.headers['Operation-Location']

    operation_key=operation_loc.split('/')[-1]
    time.sleep(3)
    result=cv_client.get_read_result(operation_key)

    if result.status==OperationStatusCodes.succeeded:
        read_res=result.analyze_result.read_results
        for analysed_res in read_res:
            for line in analysed_res.lines:
                list1.append(line.text)

In [8]:
#driver code for deciding and further proceeding whether an image is mirrored or not
def mirrorChecker(Image):
    x=0
    y=0
    for i in range(len(list1)):
        if(list1[i]=='PERSONAL DETAILS'):
            x=1
        else:
            y=y+1
    if x==1:
        if(facedetector(image_file=Image)==1):
            Data.append("Photo Accepted")
        else:
            Data.append("Photo Rejected")

    else:
        list1.clear()
        #Sending image to mirror function
        cv2.imwrite("mirror/mirrored.jpeg",mirror_this(image_file=Image))
        #Sending mirror free image to orb function
        cv2.imwrite("process/processed.jpeg",orbmaker(reference='ref/orb.jpeg',actualimage='mirror/mirrored.jpeg'))
        #Sending face detector function
        if(facedetector(image_file='process/processed.jpeg')==1):
            Data.append("Photo Accepted")
        else:
            Data.append(" Photo Rejected")
        # api  credential 
        ocrmaker('process/processed.jpeg')

    return

In [9]:
def stringappender():

#Code to convert list to string
    string1=""
    for i in range(len(list1)):
        string1=string1+list1[i]+" "

#Assigning different string values to a variable
    name="Name:"
    fname="Father's Name:"
    mname="Mother's Name:"
    dob="Date of Birth:"
    gender="Gender:"
    phone="Phone:"
    place="Place of Birth:"
    city="City:"
    state="State:"
    sign="Signature :"
    undertaking="UNDERTAKING"

#Searching whether the string value in the assigned variable is present in the string1
    Name=re.search(name,string1)
    Fname=re.search(fname,string1)
    Mname=re.search(mname,string1)
    Dob=re.search(dob,string1)
    Gender=re.search(gender,string1)
    Phone=re.search(phone,string1)
    Place=re.search(place,string1)
    City=re.search(city,string1)
    State=re.search(state,string1)
    Undertaking=re.search(undertaking,string1)

#Declaring an empty string to append the values extracted from the string1 variable
    namestring=""
    fnamestring=""
    mnamestring=""
    dobstring=""
    genderstring=""
    phonestring=""
    placestring=""
    citystring=""
    statestring=""

#piece of code to append the extracted string values from the original string named string1
    for i in range(Name.end(),Fname.start()):
        namestring=namestring+string1[i]
    Data.append(namestring)
    for i in range(Fname.end(),Mname.start()):
        fnamestring=fnamestring+string1[i]
    Data.append(fnamestring)
    for i in range(Mname.end(),Dob.start()):
        mnamestring=mnamestring+string1[i]
    Data.append(mnamestring)
    for i in range(Dob.end(),Gender.start()):
        dobstring=dobstring+string1[i]
    Data.append(dobstring)
    for i in range(Gender.end(),Phone.start()):
        genderstring=genderstring+string1[i]
    Data.append(genderstring)
    for i in range(Phone.end(),Place.start()):
        phonestring=phonestring+string1[i]
    Data.append(phonestring)
    for i in range(Place.end(),City.start()):
        placestring=placestring+string1[i]
    Data.append(placestring)
    for i in range(City.end(),State.start()):
        citystring=citystring+string1[i]
    Data.append(citystring)
    for i in range(State.end(),Undertaking.start()):
        statestring=statestring+string1[i]
    Data.append(statestring)

In [10]:
def photosender(img):

    #function call to ocr maker function
    ocrmaker(img)

    #function call to mirror checker function
    mirrorChecker(img)

    #function call to append the converted text into string and further manipulation
    stringappender()

In [11]:
#Send your document image here
photosender('applicant/sam.jpeg')

In [12]:
print(Data)

['Photo Accepted', ' R. lokulprasath ', ' V. Raguraman ', ' R. Anbarasi ', ' 10/07/2002 ', ' Male ', ' 9943170426 ', ' Puducherry ', ' Puducherry ', ' Puducherry ']
